# 05 Related vs unrelated transfer
Hypothesis: related source-target pairs transfer better than unrelated pairs.

## Step 1: Imports and setup
Run the same adaptation strategy on two task pairings.

In [ ]:
from pathlib import Path
import sys
import torch
import pandas as pd
import matplotlib.pyplot as plt

ROOT = Path.cwd().resolve()
while ROOT != ROOT.parent and not (ROOT / 'src').is_dir():
    ROOT = ROOT.parent
sys.path.insert(0, str(ROOT / 'src'))

from utils.seed import set_seed
from data.cifar10_transfer import get_cifar10_transfer
from models.transfer_resnet import TransferResNet18
from methods.transfer_learning import pretrain_source, build_transferred_model, run_target_adaptation

FIGS = ROOT / 'outputs' / 'figures'
FIGS.mkdir(parents=True, exist_ok=True)

## Step 2: Define a single-scenario runner
Keep the workflow explicit: pretrain source, then gradual unfreeze on target.

In [ ]:
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

def run_gradual_scenario(seed, source_classes, target_classes, target_train_per_class, lr_target, target_epochs, schedule):
    set_seed(seed)
    loaders = get_cifar10_transfer(
        data_dir='./data',
        source_classes=source_classes,
        target_classes=target_classes,
        source_train_per_class=1000,
        source_test_per_class=300,
        target_train_per_class=target_train_per_class,
        target_test_per_class=300,
        probe_per_class=120,
        batch_size=128,
        num_workers=2,
        seed=seed,
    )

    source_model = TransferResNet18(num_classes=loaders.source_num_classes)
    pretrain_source(
        model=source_model,
        train_loader=loaders.source_train,
        test_loader=loaders.source_test,
        device=DEVICE,
        epochs=6,
        lr=0.03,
        weight_decay=5e-4,
        momentum=0.9,
        use_progress=True,
    )

    model, source_head = build_transferred_model(source_model, loaders.target_num_classes)
    result = run_target_adaptation(
        model=model,
        target_train=loaders.target_train,
        target_test=loaders.target_test,
        target_probe=loaders.target_probe,
        source_test=loaders.source_test,
        source_head=source_head,
        device=DEVICE,
        strategy='gradual_unfreeze',
        epochs=target_epochs,
        lr=lr_target,
        weight_decay=5e-4,
        momentum=0.9,
        gradual_schedule=schedule,
        use_progress=True,
    )
    return pd.DataFrame(result.history)

## Step 3: Run related and unrelated scenarios
Same adaptation method, different task relatedness.

In [ ]:
related = run_gradual_scenario(
    seed=0,
    source_classes=[2, 3, 4, 5, 6, 7],
    target_classes=[3, 5, 7],
    target_train_per_class=80,
    lr_target=0.01,
    target_epochs=10,
    schedule={
        2: ['backbone.layer4'],
        5: ['backbone.layer3', 'backbone.layer2'],
        7: ['backbone.layer1', 'backbone.bn1', 'backbone.conv1'],
    },
).assign(scenario='related')

unrelated = run_gradual_scenario(
    seed=2,
    source_classes=[0, 1, 8, 9],
    target_classes=[3, 5, 7],
    target_train_per_class=60,
    lr_target=0.02,
    target_epochs=12,
    schedule={
        3: ['backbone.layer4'],
        6: ['backbone.layer3', 'backbone.layer2'],
        9: ['backbone.layer1', 'backbone.bn1', 'backbone.conv1'],
    },
).assign(scenario='unrelated')

both = pd.concat([related, unrelated], ignore_index=True)
both.head()

## Step 4: Plot scenario curves
Compare target quality and representation stability.

In [ ]:
fig, ax = plt.subplots(figsize=(6.5, 3.6))
for scenario, df in both.groupby('scenario'):
    ax.plot(df['epoch'], df['target_test_acc'], marker='o', label=scenario)
ax.set_title('Related vs unrelated target accuracy')
ax.set_xlabel('epoch')
ax.set_ylabel('target_test_acc')
ax.grid(alpha=0.25)
ax.legend(frameon=False)
fig.savefig(FIGS / '05_related_vs_unrelated_target_acc.png', dpi=150, bbox_inches='tight')

fig, ax = plt.subplots(figsize=(6.5, 3.6))
for scenario, df in both.groupby('scenario'):
    ax.plot(df['epoch'], df['feature_drift'], marker='o', label=scenario)
ax.set_title('Related vs unrelated feature drift')
ax.set_xlabel('epoch')
ax.set_ylabel('feature_drift')
ax.grid(alpha=0.25)
ax.legend(frameon=False)
fig.savefig(FIGS / '05_related_vs_unrelated_feature_drift.png', dpi=150, bbox_inches='tight')

In [ ]:
summary = both.groupby('scenario').agg(
    final_target_acc=('target_test_acc', 'last'),
    final_retention=('source_retention_acc', 'last'),
    final_drift=('feature_drift', 'last'),
).reset_index()
summary

### Expected Outcome
Related transfer should produce better target accuracy with lower drift.

## Interpretation
Relatedness shifts the full learning trajectory, not only the endpoint.